In [1]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer

import ast
import numpy as np

In [2]:
path = '/workspaces/prediction_house_price_from_airbnb/Data/'

In [3]:
df01=pd.read_parquet(path=path +'bronze/pontos_interesses_complete.parquet')

In [4]:
df=pd.read_parquet(path=path+'silver/listings.parquet')

In [5]:

# Substitui caracteres indesejados
df['amenities'] = df['amenities'].str.replace("[", "").str.replace("]", "").str.replace("\"", "")

# Divide a coluna 'amenities' em várias colunas
amenities_list = df['amenities'].str.split(',').tolist()

# Cria um novo DataFrame com as amenities divididas
new_df = pd.DataFrame(amenities_list)

# Codifica as colunas como booleanas
new_df = new_df.notnull()

# Aplica uma função lambda que transforma True em 1 e False em 0
new_df = new_df.applymap(lambda x: 1 if x == True else 0)


In [6]:
df.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,distance
0,783493769216852616,https://www.airbnb.com/rooms/783493769216852616,20221229002515,2022-12-29,city scrape,"Leme, Brasil",Apartamento aconchegante próximo à praia do Le...,None,https://a0.muscache.com/pictures/fe1f4b1b-a300...,491704706,...,NaN,NaN,NaN,f,2,2,0,0,NaN,6.711760
1,703973293620197060,https://www.airbnb.com/rooms/703973293620197060,20221229002515,2022-12-29,city scrape,Suíte com entrada independente em casarão 1,"Casa ampla, em excelente localização, situada ...",None,https://a0.muscache.com/pictures/miso/Hosting-...,20362236,...,4.00,5.00,NaN,t,4,0,4,0,0.26,5.701366
2,782895997622988215,https://www.airbnb.com/rooms/782895997622988215,20221229002515,2022-12-29,city scrape,Apartamento próximo ao metrô,Sua família vai estar perto de tudo ao ficar n...,None,https://a0.muscache.com/pictures/d48a6798-d1c8...,302417043,...,NaN,NaN,NaN,f,1,1,0,0,NaN,3.107676
3,23768085,https://www.airbnb.com/rooms/23768085,20221229002515,2022-12-29,city scrape,Vamos a praia,"Ótimo apartamento para duas pessoas, de frente...","Localização: praia da Barra da Tijuca, Posto 2...",https://a0.muscache.com/pictures/e43b686c-c4b4...,86611015,...,4.98,4.78,NaN,t,1,1,0,0,1.59,17.073017
4,10463735,https://www.airbnb.com/rooms/10463735,20221229002515,2022-12-29,city scrape,Sobrado aconchegante e espaçoso,"Sobrado amplo, arejado, com arquitetura inusit...",Laranjeiras é um bairro histórico do Rio de Ja...,https://a0.muscache.com/pictures/52ffaa78-2e94...,53918534,...,2.00,1.00,NaN,t,6,2,4,0,0.03,3.075010


In [24]:
df=df.merge(df01, on='id')

In [26]:
df.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,amenity_97,amenity_98,amenity_99,latitude_y,longitude_y,Pharmacy,Grocery,Tourist Attraction,Restaurant,Subway Station
0,783493769216852616,https://www.airbnb.com/rooms/783493769216852616,20221229002515,2022-12-29,city scrape,"Leme, Brasil",Apartamento aconchegante próximo à praia do Le...,None,https://a0.muscache.com/pictures/fe1f4b1b-a300...,491704706,...,0,0,0,-22.96421,-43.171600,20.0,13.0,14.0,20.0,1.0
1,703973293620197060,https://www.airbnb.com/rooms/703973293620197060,20221229002515,2022-12-29,city scrape,Suíte com entrada independente em casarão 1,"Casa ampla, em excelente localização, situada ...",None,https://a0.muscache.com/pictures/miso/Hosting-...,20362236,...,0,0,0,-22.95792,-43.182226,20.0,20.0,10.0,20.0,2.0
2,782895997622988215,https://www.airbnb.com/rooms/782895997622988215,20221229002515,2022-12-29,city scrape,Apartamento próximo ao metrô,Sua família vai estar perto de tudo ao ficar n...,None,https://a0.muscache.com/pictures/d48a6798-d1c8...,302417043,...,0,0,0,-22.93196,-43.180180,20.0,20.0,10.0,20.0,3.0
3,23768085,https://www.airbnb.com/rooms/23768085,20221229002515,2022-12-29,city scrape,Vamos a praia,"Ótimo apartamento para duas pessoas, de frente...","Localização: praia da Barra da Tijuca, Posto 2...",https://a0.muscache.com/pictures/e43b686c-c4b4...,86611015,...,0,0,0,-23.01104,-43.320340,20.0,7.0,3.0,20.0,0.0
4,10463735,https://www.airbnb.com/rooms/10463735,20221229002515,2022-12-29,city scrape,Sobrado aconchegante e espaçoso,"Sobrado amplo, arejado, com arquitetura inusit...",Laranjeiras é um bairro histórico do Rio de Ja...,https://a0.muscache.com/pictures/52ffaa78-2e94...,53918534,...,0,0,0,-22.93555,-43.191070,13.0,8.0,6.0,20.0,0.0


In [27]:
df.to_parquet(path=path+'gold/listings_pontos.parquet')